[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Integración de ORM con Flask-SQLAlchemy.

## La biblioteca con *SQLAlchemy*.

[*SQLAlchemy*](http://www.sqlalchemy.org/) comprende diversas herramientas enfocadas a interactuar con bases de datos relacionales de forma "pythonica".

Consta de:
* **SQLAlchemy Core**, la cual permite crear una interfaz genérica e independiente del gestor de base de datos por medio de un lenguaje  de expresiones basado en SQL.
* **SQLAlchemy ORM**, mapeador entre objetos y transacciones relacionales u ORM (por las sigas de object-relational mapper).

Conocer a profundidad la biblioteca de SQLAlchemy podría extenderse por varios capítulos, por lo que para fines de este curso, solo se explorarán superficialmente el ORM de la extensión [*Flask-SQLAlchemy*](http://flask-sqlalchemy.pocoo.org) y la extensión de Jupyter del lenguaje de expresiones de SQLAlchemy ([*ipython-sql*](https://github.com/catherinedevlin/ipython-sql)). 

## La extensión *Flask-SQLAlchemy*.

La extensión *Flask-SQLAlchemy* permite realizar operaciones de gestión de bases de datos desde todos los contextos de Flask de forma segura.

In [ ]:
!pip install flask-sqlalchemy

### La clase *flask_sqlalchemy.SQLAlchemy*.

Esta clase contiene todas las funcionalidades de SQLAlchemy.

La sintaxis para poder usarla junto con una aplicación de Flask es la siguiente:

```
from flask_sqlalchemy import SQLAlchemy
...
...
<nombre de la conexión a la base de datos> = SQLAlchemy(<objeto tipo Flask>)
```

### Configuración de la aplicación del objeto tipo *Flask* para *Flask-SQLAlchemy*.

Para poder integrar la extensión a un aplicación de Flask, es necesario añadir ciertos campos al atributo *config* de un objeto instanciado de la clase *flask.Flask*.

La lista de campos de configuración puede ser consultada en http://flask-sqlalchemy.pocoo.org/2.3/config/. Sin embargo, hay dos campos que deben de ser definidos para una operación satisfactoria.

#### Localización del gestor de la base de datos.

El campo *'SQLALCHEMY_DATABASE_URI'* le indica a la aplicación la dirección en la que se encuetra el gestor de base de datos.

```
<objeto tipo Flask>.config['SQLALCHEMY_DATABASE_URI'] = '<ruta de conexión>'
```

La ruta de conexión a la base de datos utiliza la siguiente sintaxis.

```
<dialecto>+<controlador>://<usuario>:<contraseña>@<ruta del servidor>:<puerto>/<base de datos>
```
Los dialectos son implementaciones de SQLAlchemy para cada variante de SQL de los diversos gestores de bases de datos. Por otra parte, Lls conexiones a las bases de datos se realizan por medio de controladores basados en la DBAPI de Python.
Puede consultar la lista de combinaciones ```<dialecto> + <controlador>``` soportadas por SQLALchemy en https://docs.sqlalchemy.org/en/latest/dialects/

#### Optimización de ejecución de SQLAlchemy.

El campo *'SQLALCHEMY_TRACK_MODIFICATIONS'* contiene el valor *None*, por defecto y a menos que sea requerido, debe de contener el valor *FALSE* a fin de optimizar la ejecución de la aplicación.   

## El gestor de bases de datos SQLite.

[SQLite](https://www.sqlite.org/index.html) es un gestor de bases de datos que viene incluido con Python, capaz de crear y gestionar bases de datos desde un sólo archivo e incluso en memoria.

**Nota:** Los ejemplos utilizados en este curso están basados en SQLite.

## Ejemplo de uso básico del ORM de SQLAlchemy.

### Creación de la aplicación.

Se definirá una aplicación utilizando una base de datos llamada *alumnos.db*, localizada en el subdirectorio [*data*](data). La base de datos aún no existe y será creada más adelante.

**Advertencia:** En vista de que en este capítulo se creará la base de datos, es necesario que el archivo [data/alumnos.db](data/alumnos.db) NO exista.

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/alumnos.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

El objeto *db*, al ser una instancia de *flask-sqlalchemy.SQLALchemy* es el encargado de realizar todas las operaciones de SQLAlchemy en la aplicación.

**Nota:** 
A partir de este momento cualquier referencia a *db*, es aplicable cualquier objeto instanciado de *flask-sqlalchemy.SQLALchemy*

In [ ]:
help(db)

### Definición de un modelo objeto-relacional.

Antes de crear la base de datos es necesario definir un modelo que mapee a un objeto con al menos una tabla en la base de datos.

#### La clase *db.Model*.

Es una clase la cual permite crear subclases capaces de instanciar objetos cuyos atributos estén ligados a una tabla en una base de datos.

El atributo especial *db.Model.\_\_table_name\_\_* indica el nombre de la tabla a la que los objetos instanciados estarán ligados.

#### La clase *db.Column*.

Es una clase la cual permite crear objetos que representan columnas en una tabla y que servirán como atributos para los objetos instanciados de las subclases de *db.Model*.

```
<nombre> = db.Column(<tipo de dato>, <argumentos>)
```

##### Algunos parámetros relevantes de *db.Column*. 

* El parámetro *primary\_key* con argumento igual a *True* indica que dicho atributo  y será la clave primaria de la tabla definida por db.Model.*\_\_table_name\_\_* y además será utilizado como parámetro al instanciar a la subclase de *db.Model*.
* El parámetro *unique* con argumento igual a *True* indica que no puede haber dos valores idénticos en la columna. 

##### Tipos de datos de SQLAlchemy.

Cada columna debe de ser definida en la base de datos con un tipo de dato específico.

Los tipos de datos que SQLAlchemy acepta pueden ser consultados en http://docs.sqlalchemy.org/en/latest/core/type_basics.html.



En este ejemplo utilizaremos los tipos:

* [*Integer*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.Integer).
* [*String*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.String).
* [*Float*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.Float).
* [*Boolean*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.Boolean).

A continuación se creará la clase *Alumno* que es subclase de *db.Model*, la cual estará ligada a la tabla *alumnos* cuyas columnas/atributos son:
* *cuenta* de tipo *Integer* con el argumento *primary\_key=True*.
* *nombre* de tipo *String* con tamaño de 50 caracteres.
* *primer_apellido* de tipo *String* con tamaño de 50 caracteres.
* *segundo_apellido* de tipo *String* con tamaño de 50 caracteres.
* *carrera* de tipo *String* con tamaño de 50 caracteres.
* *semestre* de tipo *Integer*.
* *promedio* de tipo *Float*.
* *al_corriente* de tipo *Boolean*. 

In [ ]:
class Alumno(db.Model):
    __tablename__ = 'alumnos'
    cuenta = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(50))
    primer_apellido = db.Column(db.String(50))
    segundo_apellido = db.Column(db.String(50))
    carrera = db.Column(db.String(50))
    semestre = db.Column(db.Integer)
    promedio = db.Column(db.Float)
    al_corriente = db.Column(db.Boolean)

### Creación de las tablas de la base de datos con el método *create_all()*.

Una base de datos puede consistir de múltiples tablas. En este caso, la base de datos sólo contendrá a la tabla *alumnos* ligadas a la clase *Alumnos*.

Para crear la base de datos con las tablas definidas se utiliza el método *db.create_all()*. 

* En caso de que no exista el archivo con la base de datos, este será creado.
* En caso de que ya existan tablas definidas en la base de datos, sólo se crearán las que sean nuevas y los datos que ya contienen no serán eliminados.

In [ ]:
db.create_all()

El archivo *alumnos.db* fue creado en el directorio [*data*](data/).

A partir de este momento, cada objeto instanciado de *Alumno* puede ser representado como un registro de la tabla *alumnos*.

### Operaciones en la base de datos con el objeto *db.session*.

El objeto *db.sessions* contiene atributos y métodos que permiten interactuar con la base de datos. En este y el próximo capítulo usaremos:

* El método *db.session.add()* que añade o sustituye el regisrto ligado al objeto instanciado de una subclase de *db.Model* en el registro correspondiente dentro de la base de datos.
* El método *db.session.delete()* que elimina el registro ligado al objeto.
* El método *db.session.commit()* el cual aplica los cambios en la base de datos.

###  Poblamiento de la base de datos a partir de un archivo.

El archivo [*data/alumnos.txt*](data/alumnos.txt) contiene la representación de un objeto tipo *list* que a su vez contiene objetos tipo *dir* con los campos:

* *'Cuenta'*.
* *'Nombre'*.
* *'Primer Apellido'*.
* *'Segundo Apellido'*.
* *'Carrera'*.
* *'Semestre'*.
* *'Promedio'*.
* *'Al Corriente'*.

Se puede observar que los identificadores de los campos son similares a los nombres de las columnas definidas en la clase *Alumnos*. Sólo es necesario convertirlos a minúsculas y sustituir los espacios con guiones bajos.

La siguiente celda convertirá cada objeto tipo *dict* del archivo *data/alumnos.txt* en un objeto instaciado de *Alumno* y lo ligará a la base de datos.

Cada objeto ligado a *alumno* será sustituido por el siguiente cada vez que *Alumno* sea instanciado en cada iteración. Esto implica que el objeto sustituído será destruido, pero sus atributos persistirán en la base de datos.

In [ ]:
# Craga los datos del archivo.
with open('data/alumnos.txt', 'tr') as archivo:
    base = eval(archivo.read())
    
# Puebla la tabla alumno instanciando de Alumno a cada elemento de tipo _dict_.
for registro in base:
    # Se instancia el objeto alumno a partir de la clase Alumno.
    alumno  = Alumno(cuenta=registro['Cuenta'])
    del registro['Cuenta']
    for campo in registro:
        # Los campos son convertidos en atributos. 
        setattr(alumno, campo.lower().replace(' ', '_'), registro[campo])
    #
    db.session.add(alumno)
# Los cambios se aplican a la base de datos.
db.session.commit()

### Consultas mediante el ORM.

Las subclases de *db.Model* pueden realizar búsquedas relacionadas a los objetos instanciados a partir de ellas y de las tablas ligadas a ellas.

SQLAlchemy puede realizar consultas tanto mediante álgebra relacional como de búsqueda sobre los atributos de sus objetos instanciados.

**Nota:** Para ilustrar estas operaciones se utilizará la clase *Alumno*, la cual es subclase de *db.Model*.

#### El objeto *Alumno.query*.

El objeto *Alumno.query* es una instancia de la clase *flask_sqlalchemy.BaseQuery* y contiene los métodos de búsqueda de SQLAlchemy y algunos adicionales diseñados para ser utilizados por Flask.

Para efectos de este curso sólo se explorarán los siguientes métodos:

* *query.first()* regresa al primer objeto encontrado en una búsqueda.
* *query.all()* regresa un objeto tipo *list* con todos los objetos resultantes de una búsqueda.
* *query.filter()* regresa un objeto de tipo *Query* con los objetos encontrados al ejecutar una búsqueda que satisfaga la expresión lógica sobre los atributos de la clase, la cual es ingresada como argumento.
* *query.filter_by()* regresa un objeto de tipo *Query* con los objetos encontrados al ejecutar una búsqueda en la tabla de la base de datos cuyos valores en la columna sean iguales al valor que se ingresa como argumento en la forma ```<columna>=<valor>```.

**Ejemplos:**

In [ ]:
Alumno.query.filter(Alumno.cuenta)

In [ ]:
Alumno.query.filter(Alumno.cuenta).all()

In [ ]:
Alumno.query.filter(Alumno.cuenta > 1231221).all()

In [ ]:
Alumno.query.filter(Alumno.cuenta > 1231221).first()

In [ ]:
Alumno.query.filter_by(cuenta = 1231221).first()

In [ ]:
Alumno.query.filter_by(cuenta = 1231221).first().nombre

## La extensión *ipython-sql* de Jupyter.

La extensión [*ipython-sql*](https://github.com/catherinedevlin/ipython-sql) utiliza a SQLAlchemy para realizar conexiones a bases de datos y ejecutar instrucciones SQL desde una celda de Jupyer mediante "comandos mágicos".

Para cargar la extensión se debe ejecutarse desde una celda con la siguiente sintaxis:

```
%load_ext sql
```

Para conectarse a la base de datos se utiliza la siguiente sintaxis:

```
%sql <ruta a la base de datos>
```

Para ejecutar una instrucción SQL por celda se utiliza la siguiente sintaxis:

```
%sql <instrucción>
```

Para ejecutar varias instrucciones SQL en una celda se utiliza la siguiente sintaxis:

```
%%sql 
<instrucción>
<instrucción>
...
<instrucción>
```


**Ejemplo:**

* Se instalará y cargará la extensión.
* Se hará una conexión con la base de datos *data/alumnos.db*.
* Se realizarán un par de consultas SQL.

In [ ]:
!pip install ipython-sql

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///data/alumnos.db

In [ ]:
%%sql 
select * from alumnos

In [ ]:
%sql select * from alumnos where cuenta=1231221

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>